# **概要**
---
教師あり学習のタスクである**回帰**と**分類**を実行するための簡単なモデルを構築します。  
さらに、教師なし学習の一つである**クラスター分析**も行います。

# **準備**
---
まずは、必要なライブラリをインポートしておきましょう。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# **回帰タスク(電力需要予測)**
---
気象データを説明変数として、11月から4月の電力需要量を予測する機械学習モデルを構築してみましょう。

## **データの準備**
学習に使うデータをpandasのDataFrameとして読み込みます。

In [ ]:
df_pow = pd.read_csv("./power_demand.csv")  # CSVファイルの読み込み
df_pow.head(10)  # 先頭の10行を表示

読み込めたら、データの中身を確認しておきましょう。  
各カラムの意味は以下の通りです。

| カラム名 | 意味 |
| ---- | ---- |
| **month** | 月 |
| **day** | 日 |
| **temperature** | 平均気温 |
| **humidity** | 平均相対湿度 |
| **sun_hours** | 日照時間 |
| **power** | 電力需要量(万kW)|

ただし`temperature`・`humidity`・`sun_hours`については、あらかじめ平均0・分散1とする変換を行ってあります。

## **学習・精度評価**
今回は

- 単回帰分析
- 重回帰分析

の2つを試してみましょう。

### **単回帰分析**
---

#### **説明変数と目的変数に分ける**
データを説明変数と目的変数に分けます。

まずは単回帰分析を行うので、説明変数は`temperature`のみとします。  
目的変数は`power`です。

In [ ]:
X = df_pow[["temperature"]]
y = df_pow["power"]

#### **学習用データと検証用データに分割**
モデルの性能を検証するためのデータが必要になるので、あらかじめ学習用データと検証用データに分けておきます。

ここでは、

* 学習用データ：75％
* 検証用データ：25％

となるように分割します。

分割を行うには、`sklearn.model_selection`の`train_test_split`を用います。


```
train_test_split(*arrays, test_size, random_state=None)  
 - *arrays     :  分割したいデータ  
 - test_size   :  検証用データの割合
 - random_state:  乱数シード（指定しておくと実行のたびに結果が変わることがなくなる）
```
> sklearn 公式ドキュメント : [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
# ライブラリのインポート
from sklearn.model_selection import train_test_split

# 学習用データと検証用データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 学習用データと検証用データのサイズ
print(X_train.shape, X_test.shape)

#### **学習**
`sklearn.linear_model`の`LinearRegression`を用いると回帰分析を行うことができます。

`fit`メソッドで学習を行います。

In [ ]:
# ライブラリのインポート
from sklearn.linear_model import LinearRegression

# 単回帰分析を実行
lr_simple = LinearRegression()
lr_simple.fit(X_train, y_train)  # 説明変数、目的変数の順

#### **予測**
`predict`メソッドで予測を行います。

予測対象は、学習用データではなく**検証用データ**であることに注意しましょう。

In [ ]:
# 予測を実行
y_pred = lr_simple.predict(X_test)  # 検証用データを使って予測

#### **可視化**
単回帰分析の予測結果を可視化してみましょう。

横軸は説明変数(`temperature`)、縦軸は目的変数(`power`)を表しています。

In [ ]:
# 予測結果を可視化
plt.scatter(X_test, y_test, label="true")
plt.scatter(X_test, y_pred, label="pred")
plt.xlabel("temperature")
plt.ylabel("power")
plt.legend()
plt.show()

予測データが直線状になっていることから、単回帰分析は1次関数で予測していることが確認できました！

#### **精度評価**
この単回帰モデルの精度を

* MAE
* MSE
* RMSE
* RMSLE
* MAPE

で評価してみましょう。

各評価指標は`sklearn.metrics`の各種ライブラリで計算することができます。

ただしRMSE・RMSLEに関しては、MSE・MSLEの平方根を計算するという実装としています。

In [ ]:
# ライブラリのインポート
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error

# MAE(Mean Absolute Error)
print("MAE :", mean_absolute_error(y_test, y_pred))  # 真の値、予測値の順

# MSE(Mean Squared Error)
print("MSE :", mean_squared_error(y_test, y_pred))

# RMSE(Root Mean Squared Error)
print("RMSE :", mean_squared_error(y_test, y_pred)**0.5)  # RMSEはMSEの平方根

# RMSLE(Root Mean Squared Logarithmic Error)
print("RMSLE :", mean_squared_log_error(y_test, y_pred)**0.5)  # RMSLEはMSLEの平方根

# MAPE(Mean Absolute Percentage Error)
print("MAPE :", mean_absolute_percentage_error(y_test, y_pred))

### **重回帰分析**
---

#### **説明変数と目的変数に分ける**
データを説明変数と目的変数に分けます。

今度は説明変数を

* `temperature`
* `humidity`
* `sun_hours` 

の3つに増やして予測を行っていきます。

目的変数は単回帰分析と変わらず`power`です。

In [ ]:
# 説明変数
X = df_pow[["temperature","humidity","sun_hours"]]
# 目的変数
y = df_pow["power"]

#### **学習用データと検証用データに分割**
学習用データと検証用データに分けます。

先ほどと同様に、`train_test_split`を用いて分割します。

In [ ]:
# 学習用データと検証用データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 学習用データと検証用データのサイズ
print(X_train.shape, X_test.shape)

#### **学習**
重回帰分析でも単回帰分析と同様、`sklearn.linear_model`の`LinearRegression`を用いることができます。

In [ ]:
# 重回帰分析を実行
lr_multi = LinearRegression()
lr_multi.fit(X_train, y_train)

#### **予測**
`predict`メソッドで予測を行います。


In [ ]:
# 予測を実行
y_pred = lr_multi.predict(X_test)
print(y_pred)

#### **可視化**
重回帰分析の予測結果を可視化してみましょう。

単回帰分析の時は横軸を説明変数・縦軸を目的変数として可視化できましたが、今回のような重回帰分析の場合は説明変数が複数存在するため説明変数と目的変数のグラフを作ることが難しいです。

そのため、横軸に正解（実際の電力需要量）、縦軸に予測値（予測した電力需要量）として散布図をプロットしてみましょう。

`y=x`の直線が完璧に予測できているパターンなので、それに近いほどうまく予測できていることになります。

In [ ]:
# 予測結果を可視化
plt.plot(y_test, y_test, label="理想y=x")
plt.scatter(y_test, y_pred, color="orange", label="予測")
plt.xlabel("実際の電力需要量[万kW]")
plt.ylabel("予測電力需要量[万kW]")
plt.legend()
plt.show()

#### **精度評価**
単回帰の場合と同様に、重回帰モデルの精度を

* MAE
* MSE
* RMSE
* RMSLE
* MAPE

で評価してみます。

In [ ]:
# MAE(Mean Absolute Error)
print("MAE :", mean_absolute_error(y_test, y_pred))

# MSE(Mean Squared Error)
print("MSE :", mean_squared_error(y_test, y_pred))

# RMSE(Root Mean Squared Error)
print("RMSE :", mean_squared_error(y_test, y_pred)**0.5)

# RMSLE(Root Mean Squared Logarithmic Error)
print("RMSLE :", mean_squared_log_error(y_test, y_pred)**0.5)

# MAPE(Mean Absolute Percentage Error)
print("MAPE :", mean_absolute_percentage_error(y_test, y_pred))

単回帰モデルと比べるといずれの評価指標も小さくなっており、精度が高くなっていることがわかります。

#### **係数の解釈**
重回帰分析では、次のような式で予測を行います。

$$
y = w_1x_1 + w_2x_2 + w_3x_3
$$

ここで、$w_1,w_2,w_3$はデータから学習することで決定される係数です（回帰係数という）。  
この回帰係数を見ることで、説明変数が目的変数にどの程度影響を与えるのかを知ることができます。

次のように`coef_`属性を使うと、回帰係数を取得することができます。


In [ ]:
print(lr_multi.coef_)

左から順に、`temperature`、`humidity`、`sun_hours`の回帰係数が表されています。

`temperature`の係数の絶対値が最大であることから、3つの説明変数のうち最も目的変数に影響を与える変数は`temperature`であることが考えられます。

また、回帰係数の正負に注目すると、すべての変数が負の数になっています。  
これは、`temperature`、`humidity`、`sun_hours`の値が大きくなると、目的変数`power`の値は小さくなることを示しています。

# **分類タスク(タイタニック問題)**
---
タイタニック号沈没事故の際に搭乗していた乗員乗客に関するデータを用いて、事故で生存したか死亡したかを予測する機械学習モデルを作りましょう。

## **データの準備**
学習に使うデータをpandasのDataFrameとして読み込みます。


In [ ]:
df_nic = pd.read_csv("./titanic.csv")
df_nic.head(10)

また、各カラムについては以下の通りです。

| カラム名 | 意味 |
| ---- | ---- |
| **PassengerId** | 乗客者ID |
| **Survived** | 生存状況  (0＝死亡、1＝生存) |
| **Pclass** | 旅客クラス  (1＝1等、2＝2等、3＝3等) |
| **Name** | 乗客名 |
| **Sex** | 性別 |
| **Age** | 年齢 |
| **SibSp** | 同乗している兄弟・配偶者の数 |
| **Parch** | 同乗している親・子供の数 |
| **Ticket** | チケット番号 |
| **Fare** | 旅客運賃 |
| **Cabin** | 客室番号 |
| **Embarked** | 出港地  (C＝Cherbourg、Q＝Queenstown、S＝Southampton)|

## **前処理**

#### **説明変数の選択**
過学習や次元の呪いを避けるために、目的変数に寄与しなさそうな列をあらかじめ削除しておきます。

ここでは

* `PassengerId`
* `Name`
* `Ticket`
* `Cabin`

を削除します。


In [ ]:
df_nic.drop(["PassengerId","Name","Ticket","Cabin"], axis=1, inplace=True)

`inplace=True`は、元のデータフレームを書き換えるという意味を表します。

### **カテゴリ変数を数値に置き換える**
今回のデータには数値型の値だけではなく文字列型の値も含まれています：

* `Sex`
* `Embarked`

文字列のままではモデルへ入力することができないため、適切な変換を行っておく必要があります。  

列`Sex`に対して

* `male`   → 0
* `female` → 1

列`Embarked`に対して

* `C` → 0
* `Q` → 1
* `S` → 2

と変換することも考えられますが、特に `Embarked` について、`C`と`S`の方が`C`と`Q`より離れていると考えられるのか、その倍率は2倍なのか、といった関係性を不用意に与えてしまうこととなってしまいます。

そのため、

* `Sex が female かどうかの列`
* `Sex が male かどうかの列`
* `Embarked が C かどうかの列`
* `Embarked が Qかどうかの列`
* `Embarked が S かどうかの列`

という、0, 1 のフラグ列を追加するという変換を行うことが一般的です。

これは「one-hot化」や「one-hot vector 化」と呼ばれ、下のコードで実現できます。

In [ ]:
df_nic = pd.get_dummies(df_nic)
df_nic.head(5)

#### **欠損値を埋める**
次に、欠損している値がないか確認してみましょう。  

In [ ]:
# 各列の欠損値の個数
df_nic.isnull().sum()

`Age`の列に欠損値が存在することがわかりました。  
欠損値が存在したままでは、（勾配ブ―スティングなど一部のモデルを除いて）モデルに入力することができません。

そのため欠損値は

* 中央値や最頻値などの代表値で埋める
* 欠損値を含む行や列を削除する

などの処理が必要です。ここでは

* `Age`の欠損値を中央値で埋める

という処理を行います。

In [ ]:
# 列 Age の欠損値を中央値で埋める
df_nic["Age"].fillna(df_nic["Age"].median(), inplace=True)

In [ ]:
# 欠損値がなくなったかを確認
df_nic.isnull().sum()

## **学習・精度評価**

#### **説明変数と目的変数に分ける**
データを説明変数と目的変数に分けます。

説明変数は

* `Pclass`
* `Sex_female`
* `Sex_male`
* `Age`
* `SibSp`
* `Parch`
* `Fare`
* `Embarked_C`
* `Embarked_Q`
* `Embarked_S`

目的変数は

* `Survived`

です。

In [ ]:
# 説明変数
X = df_nic[[
    "Pclass",
    "Sex_female",
    "Sex_male",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked_C",
    "Embarked_Q",
    "Embarked_S"
]]
# 目的変数
y = df_nic["Survived"]

### **学習用データと検証用データに分割**
学習用データと検証用データに分けます。  
これまでと同様に、`train_test_split`を用いて分割します。

In [ ]:
# 学習用データと検証用データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 学習用データと検証用データのサイズ
print(X_train.shape, X_test.shape)

### **ロジスティック回帰**
---


#### **学習**
`sklearn.linear_model`の`LogisticRegression`を用いるとロジスティック回帰を行うことができます。

`fit`メソッドで学習を行います。

In [ ]:
# ライブラリのインポート
from sklearn.linear_model import LogisticRegression

# ロジスティック回帰の実行
logit = LogisticRegression(max_iter=500)
logit.fit(X_train, y_train)

#### **予測**
`predict`メソッドで予測を行います。ここで返される値は`0`または`1`のどちらかです。

一方、`predict_proba`メソッドを使った場合は、目的変数が各クラスに属する確率をそれぞれ返します。

In [ ]:
# predictの場合は 0 or 1
y_pred = logit.predict(X_test)
print(y_pred)

In [ ]:
# predict_probaの場合は確率(0～1)
y_pred_p = logit.predict_proba(X_test)[:,1]  # 目的変数が1となる確率を取り出す
print(y_pred_p)

#### **精度評価**
このロジスティック回帰モデルの精度を

* 正解率
* 再現率
* 適合率
* F値
* AUC

で評価してみましょう。

各評価指標は`sklearn.metrics`の各種ライブラリで計算することができます。

In [ ]:
# ライブラリのインポート
from sklearn.metrics import confusion_matrix

# 混同行列
print(confusion_matrix(y_test, y_pred))

【補足】
`confusion_matrix` の各要素は
* 左上
    * 実際に0（死亡）で、0と予測した数(True Negative)
* 右上
    * 実際に0（死亡）で、1と予測した数(False Positive)
* 左下
    * 実際に1（生存）で、0と予測した数(False Negative)
* 右下
    * 実際に1（生存）で、1と予測した数(True Positive)
で並んでいる。

$$
\begin{matrix}
 & \mathrm{predict\_as\_0} & \mathrm{predict\_as\_1} \\
\mathrm{actual\_0} & TN & FP \\
\mathrm{actual\_1} & FN & TP \\
\end{matrix}
$$

In [ ]:
# ライブラリのインポート
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 正解率(Accuracy)
print("Accuracy :", accuracy_score(y_test, y_pred))

# 再現率(Recall)
print("Recall :", recall_score(y_test, y_pred))

# 適合率(Precision)
print("Precision :", precision_score(y_test, y_pred))

# F値(F-value)
print("F-value :", f1_score(y_test, y_pred))

In [ ]:
# ライブラリのインポート
from sklearn.metrics import roc_auc_score, roc_curve

# AUC
print("AUC :", roc_auc_score(y_test, y_pred_p))

# ROC曲線
fpr,tpr,thresholds = roc_curve(y_test, y_pred_p)

# 可視化
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

### **SVM(Support Vector Machine)**
---

#### **学習**
`sklearn.svm`の`SVC`を用いるとSVMによる分類を行うことができます。  
SVMのハイパーパラメータである`C`（誤分類に対して与えるペナルティの大きさ）を指定し、`fit`メソッドで学習を行います。

In [ ]:
# ライブラリのインポート
from sklearn.svm import SVC

# SVCの実行
svc = SVC(C=750, probability=True)  # Cパラメータを750とする
svc.fit(X_train, y_train)

#### **予測**
`predict`メソッドで予測を行います。ここで返される値は`0`または`1`のどちらかです。

一方、`predict_proba`メソッドを使った場合は、目的変数が各クラスに属する確率をそれぞれ返します。

In [ ]:
# predictの場合は 0 or 1
y_pred = svc.predict(X_test)
print(y_pred)

In [ ]:
# predict_probaの場合は確率(0～1)
y_pred_p = svc.predict_proba(X_test)[:,1]  # 目的変数が1となる確率を取り出す
print(y_pred_p)

#### **精度評価**

In [ ]:
# 混同行列
print(confusion_matrix(y_test, y_pred))

In [ ]:
# 正解率(Accuracy)
print("Accuracy :", accuracy_score(y_test, y_pred))

# 再現率(Recall)
print("Recall :", recall_score(y_test, y_pred))

# 適合率(Precision)
print("Precision :", precision_score(y_test, y_pred))

# F値(F-value)
print("F-value :", f1_score(y_test, y_pred))

In [ ]:
# AUC
print("AUC :", roc_auc_score(y_test, y_pred_p))

# ROC曲線
fpr,tpr,thresholds = roc_curve(y_test, y_pred_p)

# 可視化
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

### **k近傍法(k-NN)**
---

##### **学習**
`sklearn.neighbors`の`KNeighborClassifier`を用いるとk近傍法による分類を行うことができます。  
ハイパーパラメータ`k`の値を指定し、`fit`メソッドで学習を行います。

In [ ]:
# ライブラリのインポート
from sklearn.neighbors import KNeighborsClassifier

# k近傍法の実行
knn = KNeighborsClassifier(n_neighbors=3)  # k=3とする
knn.fit(X_train, y_train)

#### **予測**
`predict`メソッドで予測を行います。ここで返される値は`0`または`1`のどちらかです。

一方、`predict_proba`メソッドを使った場合は、目的変数が各クラスに属する確率をそれぞれ返します。

In [ ]:
y_pred = knn.predict(X_test)
y_pred_p = knn.predict_proba(X_test)[:,1]

In [ ]:
print(y_pred)

In [ ]:
print(y_pred_p)

#### **精度評価**

In [ ]:
# 混同行列
print(confusion_matrix(y_test, y_pred))

In [ ]:
# 正解率(Accuracy)
print("Accuracy :", accuracy_score(y_test, y_pred))

# 再現率(Recall)
print("Recall :", recall_score(y_test, y_pred))

# 適合率(Precision)
print("Precision :", precision_score(y_test, y_pred))

# F値(F-value)
print("F-value :", f1_score(y_test, y_pred))

In [ ]:
# AUC
print("AUC :", roc_auc_score(y_test, y_pred_p))

# ROC曲線
fpr,tpr,thresholds = roc_curve(y_test, y_pred_p)

# 可視化
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

# **クラスター分析(顧客セグメンテーション)**
---
顧客の属性データを用いて、顧客をいくつかのグループに分ける教師なし学習モデルを作りましょう。

## **データの準備**
学習に使うデータをpandasのDataFrameとして読み込みます。

データ出典：https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python

In [ ]:
df_cust = pd.read_csv("./Mall_Customers.csv")
df_cust.head(10)

また、各カラムについては以下の通りです。

| カラム名 | 意味 |
| ---- | ---- |
| **CustomerID** | 顧客のID |
| **Gender** | 顧客の性別 |
| **Age** | 顧客の年齢 |
| **Annual Income (k$)** | 顧客の年収 |
| **Spending Score (1-100)** | 顧客の行動と支出から割り当てられたスコア |

今回は、これらのうち

* `Annual Income (k$)`
* `Spending Score (1-100)`

の2つのみを使ってモデルを作っていきます。

In [ ]:
x = "Annual Income (k$)"
y = "Spending Score (1-100)"

sns.scatterplot(x=x, y=y, data=df_cust)
plt.show()

## **前処理**

### **スケーリング**
k-means法では各データ間の距離を用いるため、説明変数間でスケールが統一されていないとうまくいかない場合があります。

ここでは、最大値が1、最小値が0になるようにデータを変換する**正規化**という手法を用いてスケールを統一します。

$$
(変換後) = \frac{(\text{変換前})-(\text{最小値})}{(\text{最大値})-(\text{最小値)}}
$$

`sklearn.preprocessing`の`MinMaxScaler`を用いると正規化を行うことができます。

In [ ]:
# ライブラリのインポート
from sklearn.preprocessing import MinMaxScaler

# 必要な列を取り出す
X = df_cust[["Annual Income (k$)", "Spending Score (1-100)"]]

# 正規化
minmax = MinMaxScaler()
X = minmax.fit_transform(X)

## **k-means法**
---

#### **学習**
`sklearn.cluster`の`KMeans`を用いるとk-means法によるクラスター分析を行うことができます。  

引数`n_clusters`でクラスター数を指定し、`fit`メソッドで学習を行います。


In [ ]:
# ライブラリのインポート
from sklearn.cluster import KMeans

# k-meansの実行
km = KMeans(n_clusters=5)  # クラスター数を5とする
km.fit(X)

#### **予測**
`predict`メソッドで各データがクラスターに割り当てられます。  
今回はクラスター数を5としたので、各データには`0`から`4`までのラベルが割り当てられます。

In [ ]:
labels = km.predict(X)
print(labels)

### **可視化**
うまくクラスターに分割できているかを確認してみましょう。

In [ ]:
x = "Annual Income (k$)"
y = "Spending Score (1-100)"

df_lab = pd.DataFrame(np.c_[X, labels], columns=[x,y,"cluster"])

sns.scatterplot(x=x, y=y, hue="cluster", data=df_lab, palette="Set1")
plt.show()

5つのクラスターにうまく分類できていることがわかります。